In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH21-AGENT")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH21-AGENT


# React Agent 

In [3]:
import json
from langchain_openai import ChatOpenAI
from langchain_teddynote.tools.tavily import TavilySearch
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

## 메모리 설정 

In [4]:
# 메모리 설정
memory = MemorySaver()

# 모델 설정
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## 도구 설정 
### 웹 검색 

In [5]:
web_search = TavilySearch(
    topic="general",  # 뉴스 주제 (general 또는 news)
    include_domains=["google.com"],
    max_results=5,  # 최대 검색 결과
    include_answer=False,
    include_raw_content=False,
    include_images=False,
    format_output=False,
)

web_search.name = "web_search"
web_search.description = (
    "Use this tool to search on the web for any topic other than news."
)

In [6]:
result = web_search.search("AI 관련된 최신 뉴스를 검색해줘")
result

[{'url': 'https://cloud.google.com/use-cases/free-ai-tools?hl=ko',
  'title': '2025년에 무료로 사용할 수 있는 10가지 이상의 AI 도구',
  'content': 'Google Cloud의 무료AI도구.최신 AI모델을 사용하고 무료AI도구로 아이디어를앱으로 구현. GoogleAIStudio는 개발자, 학생, 연구자가 Gemini 모델을 사용해',
  'score': 0.23651665,
  'raw_content': None},
 {'url': 'https://news.google.com/home?hl=ko&gl=KR&ceid=KR:ko',
  'title': 'Google 뉴스',
  'content': '뉴스 Google 뉴스 * Google 정보 * Android 앱 다운로드 * iOS 앱 다운로드 * 의견 보내기 * 언어 및 지역한국어 (대한민국) Google 날씨 ### 주요 뉴스 \'북·중·러 정상 한자리\' 대서특필...푸틴 차 안서 \'활짝\' 사진도 작성자: 이종원 KBS 뉴스 천안문 망루에 김정은-시진핑-푸틴 나란히…김정은에 최고수준 예우 작성자: 김민정 북러 정상 베이징 회담…“전망적 협력 논의, 국제문제 의견 교환” 강미정 조국혁신당 대변인 "당이 성추행 피해자 절규 외면 …오늘 탈당" MBC 뉴스 ["믿었던 동지들이‥성폭력에, 2차 가해" 눈물 쏟으며 "탈당" [현장영상]](./read/CBMieEFVX3lxTE45SDJXbVQ3cXBHU1oxZ255eXZoUnRRZnJCaXlnc0FLS3JtWVhETjVUYlhMNDl1c1hpaWtnZlRTZDZWSmhXRlJxNlJxMHgzTlUtQkNub2hJYVl5aG4wakU4c2dLRkNsVHpFZndpcEExcEllVlk5TVRyVtIBeEFVX3lxTE4xQXUzdWN2NzlHcmlDSExQeEVCd1d1VWlTbk9xeWZWUXN1SXBZanptaXkwWkg0RF91TDRiQlVGcWMtdW5hN1FhNUtBQzBMRzV

### 구글 뉴스 키워드 검색 

In [7]:
from langchain_teddynote.tools import GoogleNews
from langchain.tools import tool
from typing import List, Dict

In [8]:
# 키워드로 뉴스 검색하는 도구 정의 
@tool
def search_news_by_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=10)

In [9]:
search_news_by_keyword.invoke("AI 관련된 최신 뉴스를 검색해줘")

AI 관련된 최신 뉴스를 검색해줘


[{'url': 'https://news.google.com/rss/articles/CBMiYkFVX3lxTFBWakp0aHdNN2VrM3RjY0JyV0lReFprYTFsZkRqc2F3M3J0UVRTYkdvWm9WV1FheHZqSHA4MlNrQWpudmRZcFB2NVdBWVlaUjRkZExNWWVXWnplY2g0WUlGb1hR?oc=5',
  'content': "'한국 사회 내 부정선거 음모론 설명'... 언론사 AI에 물어보니 - 한국기자협회"},
 {'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTFBfWVk0cW5zVklfNWhZdUpqMlhENnI1UzNQS3BxeHpPM2JucGppV2wwM3NYZ1lDSVRGRVRMT0U0b0NOMkNicDBzYmxSdjhBQk5LcGhMRjhn?oc=5',
  'content': '현대백화점, 한화파워시스템 부산에 7500억원 투자...2200명 신규 고용 - 매일경제'},
 {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE5SOUJPa0F6eTRmMHVIdkpPRmNyNm5BQlJGbzBqaFRIYWlmb2R5U3ZqRTBfaUxfbGNxUW04cXpvdGJidXhwZUNWbDJXQlpmSnRkY3lIbzFXdnNIUk9YXzlSS1JockNzb21rTnc?oc=5',
  'content': "'빅카인즈 AI'로 쏟아지는 뉴스 한번에 분석 - 대한민국 정책브리핑"},
 {'url': 'https://news.google.com/rss/articles/CBMiTkFVX3lxTE94YnA3amhDMW5TZkFoYXdGaXJtTkRxMjExejBISXBlcnRwWjlxS1RxTGNIM2pOd1NuYTBDRFhOWHZPWDRnbzRJdWV3aS1Fdw?oc=5',
  'content': 'Perplexity AI: AI 검색 엔진의 혁신 - 브런치'},
 {'url': 'https://news.google

### 파일 관리

In [10]:
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_core.documents import Document

In [11]:
# 작업 디렉토리 설정
working_directory = "temp"

# FileManagementToolkit 객체 생성
file_management_tools = FileManagementToolkit(
    root_dir=str(working_directory),
).get_tools()

In [12]:
file_management_tools

[CopyFileTool(root_dir='temp'),
 DeleteFileTool(root_dir='temp'),
 FileSearchTool(root_dir='temp'),
 MoveFileTool(root_dir='temp'),
 ReadFileTool(root_dir='temp'),
 WriteFileTool(root_dir='temp'),
 ListDirectoryTool(root_dir='temp')]

In [13]:
@tool
def load_pdf(file_path: str) -> List[Document]:
    """Use this tool to load a PDF file and return the content"""
    loader = PDFPlumberLoader(file_path)
    return loader.load()

In [14]:
# 도구 목록 정의 
# * 역할 : 리스트나 튜플을 풀어서 넣는 역할 
# file_management_tools를 리스트 통째로 넣는 것이 아니라, 그 안의 요소들을 꺼내서 tools 리스트에 합쳐주는 역할 
tools = [web_search, search_news_by_keyword, *file_management_tools, load_pdf]
tools

[TavilySearch(name='web_search', description='Use this tool to search on the web for any topic other than news.', client=<tavily.tavily.TavilyClient object at 0x1282ee490>, include_domains=['google.com'], max_results=5, include_raw_content=False),
 StructuredTool(name='search_news_by_keyword', description='Look up news by keyword', args_schema=<class 'pydantic.v1.main.search_news_by_keywordSchema'>, func=<function search_news_by_keyword at 0x129388040>),
 CopyFileTool(root_dir='temp'),
 DeleteFileTool(root_dir='temp'),
 FileSearchTool(root_dir='temp'),
 MoveFileTool(root_dir='temp'),
 ReadFileTool(root_dir='temp'),
 WriteFileTool(root_dir='temp'),
 ListDirectoryTool(root_dir='temp'),
 StructuredTool(name='load_pdf', description='Use this tool to load a PDF file and return the content', args_schema=<class 'pydantic.v1.main.load_pdfSchema'>, func=<function load_pdf at 0x12938b2e0>)]

### Agent 생성 

In [15]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

정의한 에이전트를 시각화 하여 확인해 보면...

In [16]:
from langchain_teddynote.graphs import visualize_graph

visualize_graph(agent_executor)

[ERROR] Visualize Graph Error: Failed to render the graph using the Mermaid.INK API. Status code: 204.


In [18]:
print(agent_executor.get_graph().draw_ascii())

        +-----------+         
        | __start__ |         
        +-----------+         
              *               
              *               
              *               
          +-------+           
          | agent |           
          +-------+           
         *         .          
       **           ..        
      *               .       
+-------+         +---------+ 
| tools |         | __end__ | 
+-------+         +---------+ 


### 출력 함수 정의 

In [19]:
def stream_agent(instruction, agent_executor, config, print_tools=False):
    # 질문 입력
    inputs = {"messages": [("human", instruction)]}

    # 동기 스트림 처리(stream_mode="messages")
    # chunk_msg: 실시간 출력 메시지, metadata: 노드 정보
    for chunk_msg, metadata in agent_executor.stream(
        inputs, config, stream_mode="messages"
    ):
        if metadata["langgraph_node"] == "agent":
            if chunk_msg.content:
                print(chunk_msg.content, end="", flush=True)
        elif metadata["langgraph_node"] == "tools":
            if print_tools:
                print(chunk_msg.content)

In [20]:
# Config 설정
config = {"configurable": {"thread_id": "abc123"}}

In [21]:
stream_agent("안녕? 내 이름은 김영희야. 만나서 반가워요.", agent_executor, config)

안녕하세요, 김영희님! 만나서 반갑습니다. 어떻게 도와드릴까요?

In [22]:
config = {"configurable": {"thread_id": "abc123"}}
stream_agent("제가 누구라고 했지요?", agent_executor, config)

김영희님이라고 하셨습니다. 맞나요?

# 결과 확인 
## 뉴스 검색 및 보고서 작성 

In [23]:
instruction = """
다음의 내용을 순서대로 처리해 주세요. 
1. 에이전트 관련된 최신 뉴스를 검색 합니다. 
2. 에이전트 활용방법(LangGraph)에 대한 예제를 조사해 주세요. 
3. 최신 뉴스와 예제를 참고하여 보고서를 작성해 주세요.
4. 작성한 보고서의 내용만 파일로 저장해 주세요. (파일 이름은 agent_report.md 입니다.)
"""

In [24]:
stream_agent(instruction, agent_executor, config, print_tools=True)

에이전트
[{"url": "https://news.google.com/rss/articles/CBMiX0FVX3lxTE5ha085ZW05S0dISzQ4dFRMUU9zbmw1cHJEQjZMbFplLXB0Yl9oTFJxeGNydVQybG9sQ3JfZnJaZ0h1SEtOY0xoRHN4bV9XeGdDN0IyYUxDeXc5NENDU09N?oc=5", "content": "[AI와 보안] AI 에이전트 ‘통합리모콘’ 같은 ‘MCP’...F5 “권한 관리 등 통합보안 필요” - 보안뉴스"}, {"url": "https://news.google.com/rss/articles/CBMiX0FVX3lxTFBGbFliUl9PZnozTEdEZjMtUWFzUGJSR1drWEhBV2pjSmczMWJVdlVJRjNVcG42R0kyRk1aYzc5dWlvRENwQjJub0lUdWlWWTMtUGVVVkx3Yi1UVFJJeW04?oc=5", "content": "한국 그루브, 에이전트포스 파일럿 프로그램 출시 - 뉴스1"}, {"url": "https://news.google.com/rss/articles/CBMiYEFVX3lxTE1adGVNRzhmbFZDSXQ5SzNJMnVuMlVHYWdYRC1aR3pkR3dHVngxa3ZuV1JrbmxpTzVCbjZ6aHpTSHVZWkNIbjBaLUZpVGhBX0pFVTdqLUFJRnE5YmdRS1ppbg?oc=5", "content": "SK 계열사 25곳, 업무에 AI 에이전트 '에이닷 비즈' 확대 도입 - 아시아경제"}, {"url": "https://news.google.com/rss/articles/CBMiW0FVX3lxTE1DTEhhNE1IeHZfaTdMR0VHMmlXa2oxR2w5OWJ4VHdub2oxU0xiNUR1cW1PWDMzQUtrZ2xPOUhON3FjVVdnWmhmZ1I1Sk9EN1phWkpqTVJURFQ4UUHSAWBBVV95cUxNWkItZ3FRWi01Ty0xcUU5b0ZJbjJTLXdrel9xTWxUX2ZoaEFRMWFiSkJRakQ

## PDF 파일에서 필요 내용 추출/정리

In [25]:
instruction = """
data/SPRi AI Brief_Special_AI Agent_241209_F.pdf 파일을 로드하고, 목차를 출력해 주세요.
"""

In [26]:
stream_agent(instruction, agent_executor, config)

PDF 문서의 목차는 다음과 같습니다:

1. AI 에이전트(AI Agent)의 도입 및 부상
   - AI 에이전트의 정의
   - AI 에이전트의 부상 배경
   - AI 에이전트 시장 전망

2. AI 에이전트 주요 사례
   - Microsoft
   - Google
   - Apple
   - Meta
   - AWS(Amazon Web Services)
   - Salesforce
   - OpenAI
   - NVIDIA

3. AI 에이전트 관련 이슈 및 전망
   - 주요 이슈
     - 기술적 문제
     - 사회적 영향
     - 윤리적 이슈
     - 법적 이슈
   - 향후 전망

4. 참고 문헌

이 문서는 AI 에이전트에 대한 다양한 정보와 사례를 포함하고 있습니다. 추가로 궁금한 점이 있으시면 말씀해 주세요!

## 목차 생성 

In [27]:
instruction = """
이전의 작성한 목차를 참고하여 다음의 지시사항을 순서대로 처리하고, 목차를 작성해 주세요.

#지시사항
1. `에이전트` 키워드와 관련된 최신 뉴스를 검색 합니다. 
2. 1번의 내용을 바탕으로 이전에 작성한 "목차"의 형식에 맞게 목차를 작성 합니다. 
3. 완성된 목차를 파일로 저장 합니다. (파일 이름은 "agent_table_of_contents.md")

[IMPORTANT] 목차의 형식은 이전에 작성한 목차를 참고해서 작성 합니다. 
"""

In [28]:
stream_agent(instruction, agent_executor, config, print_tools=True)

에이전트
[{"url": "https://news.google.com/rss/articles/CBMiX0FVX3lxTE5ha085ZW05S0dISzQ4dFRMUU9zbmw1cHJEQjZMbFplLXB0Yl9oTFJxeGNydVQybG9sQ3JfZnJaZ0h1SEtOY0xoRHN4bV9XeGdDN0IyYUxDeXc5NENDU09N?oc=5", "content": "[AI와 보안] AI 에이전트 ‘통합리모콘’ 같은 ‘MCP’...F5 “권한 관리 등 통합보안 필요” - 보안뉴스"}, {"url": "https://news.google.com/rss/articles/CBMiX0FVX3lxTFBGbFliUl9PZnozTEdEZjMtUWFzUGJSR1drWEhBV2pjSmczMWJVdlVJRjNVcG42R0kyRk1aYzc5dWlvRENwQjJub0lUdWlWWTMtUGVVVkx3Yi1UVFJJeW04?oc=5", "content": "한국 그루브, 에이전트포스 파일럿 프로그램 출시 - 뉴스1"}, {"url": "https://news.google.com/rss/articles/CBMiYEFVX3lxTE1adGVNRzhmbFZDSXQ5SzNJMnVuMlVHYWdYRC1aR3pkR3dHVngxa3ZuV1JrbmxpTzVCbjZ6aHpTSHVZWkNIbjBaLUZpVGhBX0pFVTdqLUFJRnE5YmdRS1ppbg?oc=5", "content": "SK 계열사 25곳, 업무에 AI 에이전트 '에이닷 비즈' 확대 도입 - 아시아경제"}, {"url": "https://news.google.com/rss/articles/CBMiW0FVX3lxTE1DTEhhNE1IeHZfaTdMR0VHMmlXa2oxR2w5OWJ4VHdub2oxU0xiNUR1cW1PWDMzQUtrZ2xPOUhON3FjVVdnWmhmZ1I1Sk9EN1phWkpqTVJURFQ4UUHSAWBBVV95cUxNWkItZ3FRWi01Ty0xcUU5b0ZJbjJTLXdrel9xTWxUX2ZoaEFRMWFiSkJRakQ

--------
** End of Documents **